<a href="https://colab.research.google.com/github/hucarlos08/GEE-CIMAT/blob/main/PixelTransforms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import folium
from folium import plugins
import geemap


# Authenticate and initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project="ee-cimat")


## Folium

In [2]:
# Helper function to add Earth Engine layers to Folium maps
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Bind this method to the Folium Map class
folium.Map.add_ee_layer = add_ee_layer


## The Enhanced Vegetation Index

EVI was developed to address limitations found in the more traditional
Normalized Difference Vegetation Index (NDVI).

Key limitations of NDVI that EVI improves upon include:

1. Saturation: In areas with very dense vegetation (like tropical rainforests),
   NDVI values reach their maximum quickly and don't effectively differentiate
   between varying levels of high biomass. EVI is less prone to saturation.

2. Atmospheric Influence: NDVI is sensitive to atmospheric conditions (e.g., aerosols).
   EVI incorporates the blue band to help correct for these atmospheric effects.

3. Soil Background: NDVI values can be influenced by the brightness of the soil background,
   especially in areas with sparse vegetation. EVI includes coefficients (L, C1, C2)
   to minimize these soil background variations.

The general mathematical form of EVI is:

EVI = G * (NIR - Red) / (NIR + C1 * Red - C2 * Blue + L)

Where:
- NIR, Red, Blue are surface reflectance values for the respective bands.
- L is a canopy background adjustment factor (typically 1).
- C1, C2 are coefficients used to correct for aerosol scattering effects (typically 6 and 7.5).
- G is a gain factor (typically 2.5).

### Sentinel-2

For Sentinel-2 imagery, the specific bands corresponding to NIR, Red, and Blue
are used, and the standard coefficients (G=2.5, L=1, C1=6, C2=7.5) are applied.

The Sentinel-2 bands are:
- B8: Near-Infrared (NIR)
- B4: Red
- B2: Blue

Therefore, the EVI equation specifically for Sentinel-2 becomes:
EVI = 2.5 * (B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1)

**IMPORTANT NOTE ON SCALING:**

Sentinel-2 Surface Reflectance data in GEE ('COPERNICUS/S2_SR_HARMONIZED') often comes
scaled by 10000 (i.e., reflectance of 0.1 is stored as 1000).
The EVI formula assumes reflectance values are between 0 and 1.


In [3]:
# Define the point of interest (San Francisco Airport)
sfo_point = ee.Geometry.Point(-122.3774, 37.6194)

# Load Sentinel-2 imagery and filter by date and location
sfo_image = (ee.ImageCollection('COPERNICUS/S2')
             .filterBounds(sfo_point)
             .filterDate('2020-02-01', '2020-04-01')
             .first())

# Scale the Sentinel-2 bands appropriately
nir_scaled = sfo_image.select('B8').divide(10000)
red_scaled = sfo_image.select('B4').divide(10000)
blue_scaled = sfo_image.select('B2').divide(10000)

# Compute EVI using Earth Engine expressions
evi_expression = sfo_image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': nir_scaled,
        'RED': red_scaled,
        'BLUE': blue_scaled
    }).rename('EVI')

# Visualization parameters
evi_vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'white', 'green']}

# Interactive map display using Folium
Map = folium.Map(location=[37.6194, -122.3774], zoom_start=11)

# Add EVI layer to the map
Map.add_ee_layer(evi_expression, evi_vis_params, 'EVI - San Francisco')

# Add layer control
Map.add_child(folium.LayerControl())

# Display map
display(Map)

/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


# Burned Area Index (BAI)

## Background and Importance

The Burned Area Index (BAI) was introduced by Chuvieco et al. (2002), building on the concept proposed by Martín (1998), specifically designed to effectively highlight and delineate burned regions in satellite imagery, particularly after a wildfire event.

The index leverages the spectral response of burned areas, which typically exhibit low reflectance in the Near-Infrared (NIR) spectrum due to vegetation removal and charring, and relatively higher reflectance in the Red spectrum compared to healthy vegetation. BAI essentially calculates the spectral distance of each pixel from a reference point representing char or ash in the Red-NIR spectral space. Consequently, pixels corresponding to burnt materials, such as charcoal and ash deposits, tend to yield higher BAI values compared to unburned vegetation or soil.

## The Formula

The general equation for the Burned Area Index (BAI) is defined as:

$$ BAI = \frac{1}{( \rho_{cr} - Red )^2 + ( \rho_{cnir} - NIR )^2} $$

Where:

*   $Red$ represents the surface reflectance value (unitless, typically 0-1) in the Red portion of the spectrum.
*   $NIR$ represents the surface reflectance value (unitless, typically 0-1) in the Near-Infrared portion of the spectrum.
*   $\rho_{cr} = 0.1$ is a constant representing a reference reflectance value in the Red spectrum, characteristic of burned material.
*   $\rho_{cnir} = 0.06$ is a constant representing a reference reflectance value in the NIR spectrum, characteristic of burned material.

**Note:** The formula essentially measures the inverse squared Euclidean distance in the Red-NIR feature space between a given pixel's reflectance (`Red`, `NIR`) and a fixed reference point (`0.1`, `0.06`) representing burned conditions. Pixels closer to this reference point (i.e., burned pixels) will have a smaller denominator, resulting in a larger BAI value. It is crucial that the input `Red` and `NIR` values are actual surface reflectance (usually scaled between 0 and 1), not raw digital numbers or scaled integers.


In [4]:
# Assuming ee, folium, add_ee_layer are imported

# Define Rim Fire area point (or use a polygon)
rim_fire_point = ee.Geometry.Point(-120.083, 37.850)

# --- Use Surface Reflectance ---
collection = 'LANDSAT/LC08/C02/T1_L2'  # CORRECT ID for Landsat 8 C2 SR T1
start_date = '2013-09-15'
end_date = '2013-09-27'

# Function to apply scaling factors
def apply_scale_factors(image):
  """
  Landsat Collection 2 Surface Reflectance data ('LANDSAT/LC08/C02/T1_SR') is not provided in reflectance units (0-1)
  directly. It comes as scaled integers. You need to apply the scaling factors specified in the dataset's metadata
  before using the bands in the BAI expression. For Landsat 8/9 C2 SR, you typically need to multiply
  by 0.0000275 and add -0.2.
  """
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  # Keep thermal bands if needed, or just return optical
  return image.addBands(optical_bands, None, True)

# Retrieve and process Landsat-8 SR imagery
burn_image_sr = (ee.ImageCollection(collection)
                 .filterBounds(rim_fire_point)
                 .filterDate(start_date, end_date)
                 .filter(ee.Filter.lt('CLOUD_COVER', 20)) # Add cloud filter
                 .map(apply_scale_factors) # Apply scaling
                 .sort('CLOUD_COVER') # Sort AFTER scaling if filtering based on original metadata
                 .first())

# Check if an image was found
if burn_image_sr is None:
    print(f"No suitable image found for {start_date} to {end_date}. Adjust criteria.")
else:
    # Calculate Burned Area Index (BAI) using scaled SR bands
    # Use the SR band names (e.g., SR_B4, SR_B5)
    bai_expression_sr = burn_image_sr.expression(
        '1.0 / ((0.1 - RED)**2 + (0.06 - NIR)**2)', {
            'RED': burn_image_sr.select('SR_B4'), # Landsat 8 SR Red
            'NIR': burn_image_sr.select('SR_B5')  # Landsat 8 SR NIR
        }).rename('BAI')

    # Visualization parameters for BAI
    bai_vis_params = {'min': 0, 'max': 400, 'palette': ['white', 'yellow', 'red', 'brown', 'black']}
    #bai_vis_params = {'min': 0, 'max': 400, 'palette': ['green', 'blue', 'yellow', 'red']}


    # True-color visualization parameters for Landsat 8 SR (scaled 0-1)
    rgb_vis_params_sr = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 0.3} # Use SR band names

    # Folium map centered on Rim Fire
    Map2 = folium.Map(location=[37.850, -120.083], zoom_start=9) # Zoom out slightly maybe

    # Add True Color SR image to the map
    Map2.add_ee_layer(burn_image_sr, rgb_vis_params_sr, 'True Color SR - Rim Fire')

    # Add BAI image layer to the map
    Map2.add_ee_layer(bai_expression_sr, bai_vis_params, 'BAI (SR) - Rim Fire')

    # Add layer control
    Map2.add_child(folium.LayerControl())

    # Display map
    display(Map2)